In [118]:
!pip install tensorflow_hub

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Dell\\anaconda3\\Lib\\site-packages\\~l_dtypes\\_custom_floats.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Obtaining dependency information for tensorflow_hub from https://files.pythonhosted.org/packages/e5/50/00dba77925bf2a0a1e45d7bcf8a69a1d2534fb4bb277d9010bd148d2235e/tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tf-keras>=2.14.1 from https://files.pythonhosted.org/packages/75/aa/cf09f8956d4f276f655b13674e15d8d6015fd832f9689aa9ff2a515781ab/tf_keras-2.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow<2.17,>=2.16 from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for h5py>=3.10.0 from https://files.pythonhosted.org/packages/b6/35/ed21094

DATASET USED: 

[Khan, Muhammad Yaseen, and Nizami, Muhammad Suffian. "Urdu Sentiment Corpus (v1.0): Linguistic Exploration and Visualization of Labeled Datasetfor Urdu Sentiment Analysis." In 2020 IEEE 2nd International Conference On Information Science & Communication Technology (ICISCT). IEEE, 2020.](https://ieeexplore.ieee.org/abstract/document/9080043/)

In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
import tensorflow as tensor
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split

# Preprocessing

In [82]:
df = pd.read_csv('C:/Users/Dell/Desktop/urdu-sentiment-corpus-v1.tsv', sep='\t')
df

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P
...,...,...
995,اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے ۔,P
996,چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...,P
997,واہ جناب واہ! اچھی رہی۔ جناب خود کو فرشتہ سمجو...,P
998,اسلام آباد :پی اے ٹی کا دھرنا ختم، صفائی کے کا...,P


In [83]:
df = df.dropna()
df['Class'].unique()

array(['P', 'N', 'O'], dtype=object)

In [84]:
print('O', (df['Class'] == 'O').sum())
print('N', (df['Class'] == 'N').sum())
print('P', (df['Class'] == 'P').sum())

O 20
N 499
P 480


In [85]:
df = df[df['Class'] != 'O']
df['Class'].unique()

array(['P', 'N'], dtype=object)

In [86]:
urdu_diacritics  = ['ِ', 'ٰ', 'ُ', 'ٍ', 'ً', 'َ']
def remove_diacritics(text):
    for letter in text:
    
        if letter in urdu_diacritics:
            text = text.replace(letter, '')
    return text

In [87]:
def remove_punctuations(raw_text):
    preprocessed_text=re.sub(r'[؛۔٫٪+=@#!؟،۔)(}{]', '',raw_text)
    return preprocessed_text

In [88]:
digits = ['۶', '۴', '۵', '۸', '۲', '۰', '۷', '۹', '۳', '۱','1','2','3','4','5','6','7','8','9','0']
def remove_digits(text):
    for letter in text:
        if letter in digits:
            text = text.replace(letter, '')
    return text

In [89]:
def tokenize(text):
    tokens = text.split()
    return tokens

In [90]:
stopwords=[]
with open('C:/Users/Dell/Downloads/stop_words.txt',encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
      stopwords.append(line.strip('\n'))

stopwords

['آ',
 'آؤ',
 'آو',
 'آئی',
 'آئیں',
 'آئے',
 'آج',
 'آجاؤ',
 'آجائیں',
 'آجائیے',
 'آجاو',
 'آجکل',
 'آپ',
 'آپکا',
 'آپکو',
 'آپکی',
 'آیا',
 'اب',
 'ابھی',
 'اس',
 'اسطرح',
 'اسکا',
 'اسکی',
 'اسکے',
 'اسی',
 'اسے',
 'اطراف',
 'الگ',
 'ان',
 'انکا',
 'انکی',
 'انکے',
 'انھوں',
 'انھیں',
 'انہیں',
 'اں',
 'اور',
 'اونچا',
 'اونچی',
 'اونچے',
 'اوپر',
 'اپنا',
 'اپنی',
 'اپنے',
 'اکثر',
 'اگر',
 'اگرچہ',
 'اہم',
 'بائیں',
 'بار',
 'بارے',
 'باہر',
 'باہم',
 'بذریعہ',
 'بس',
 'بظاہر',
 'بعد',
 'بغیر',
 'بلاشبہ',
 'بند',
 'بہت',
 'بھی',
 'بیشتر',
 'بیشک',
 'بےشک',
 'تاہم',
 'تب',
 'تجھ',
 'تجھے',
 'تر',
 'ترین',
 'تعداد',
 'تلک',
 'تم',
 'تمام',
 'تمکو',
 'تمھارا',
 'تمھاری',
 'تمھارے',
 'تمھیں',
 'تمہارا',
 'تمہاری',
 'تمہارے',
 'تمہیں',
 'تو',
 'تک',
 'تھا',
 'تھی',
 'تھیں',
 'تھے',
 'تیسرا',
 'تیسری',
 'تیسرے',
 'جائیں',
 'جاتا',
 'جاتی',
 'جاتے',
 'جانا',
 'جانے',
 'جائے',
 'جب',
 'جبہی',
 'جبھی',
 'جبکہ',
 'جو',
 'جہاں',
 'جیسا',
 'جیساکہ',
 'جیسی',
 'جیسے',
 'جیسےکہ',
 'خود',
 'دا

In [91]:
def remove_stopwords(text):
    temp=[]
    words=text.split()
    for letter in words:
        if letter not in stopwords:
          if letter !='':
            temp.append(letter)
            
    return (' '.join(temp) )

In [92]:
df['Tweet'] = df['Tweet'].apply(lambda x: remove_diacritics(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_digits(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_punctuations(x))
df['Tweet'] = df['Tweet'].apply(lambda x: remove_stopwords(x))
df['Tweet']

0      ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ لکھپت ا...
1                     چندے انقلاب عمران خان وزیر اعظم بن
3      سرچ انجن گوگل نائب صدر فضا , فٹ بلندی چھلانگ ع...
4                                          لہریں یار :أْ
5      گندی زبان گٹر دماغ جاهل جیالے هو جیالا هو جاهل...
                             ...                        
995                             آدمی سالار کافی معقول ٹپ
996      چچا غالب روح معذرت ساتھہم مانا ڈیزلمفت ہاتھ برا
997             واہ جناب واہ اچھی جناب فرشتہ سمجوں انسان
998    اسلام آباد :پی اے ٹی دھرنا ختم صفائی کام آغاز ...
999                  دنیا راہ وفا ساتھتم چلے چلو چلی چلے
Name: Tweet, Length: 979, dtype: object

In [93]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stemming(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    stemmed = "".join(stemmed_words)
    return stemmed

df['Tweet'] = df['Tweet'].apply(stemming)

In [94]:
df['Tweet']

0      ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ لکھپت ا...
1                     چندے انقلاب عمران خان وزیر اعظم بن
3      سرچ انجن گوگل نائب صدر فضا , فٹ بلندی چھلانگ ع...
4                                          لہریں یار :أْ
5      گندی زبان گٹر دماغ جاهل جیالے هو جیالا هو جاهل...
                             ...                        
995                             آدمی سالار کافی معقول ٹپ
996      چچا غالب روح معذرت ساتھہم مانا ڈیزلمفت ہاتھ برا
997             واہ جناب واہ اچھی جناب فرشتہ سمجوں انسان
998    اسلام آباد :پی اے ٹی دھرنا ختم صفائی کام آغاز ...
999                  دنیا راہ وفا ساتھتم چلے چلو چلی چلے
Name: Tweet, Length: 979, dtype: object

In [95]:
labelencoder = LabelEncoder()
df['Class'] = labelencoder.fit_transform(df['Class'])
print(df['Class'])

0      1
1      0
3      1
4      1
5      0
      ..
995    1
996    1
997    1
998    1
999    1
Name: Class, Length: 979, dtype: int32


In [96]:
print('N', (df['Class'] == 0).sum())
print('P', (df['Class'] == 1).sum())

N 499
P 480


In [97]:
df.dtypes

Tweet    object
Class     int32
dtype: object

# Plain LSTM

In [98]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

X = df['Tweet']
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_seq_padded = pad_sequences(X_train_seq, 50)
X_test_seq_padded = pad_sequences(X_test_seq, 50)

In [99]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 9s 125ms/step - loss: 0.6934 - acc: 0.5054 - val_loss: 0.6925 - val_acc: 0.5306
Epoch 2/10
23/23 [==============================] - 1s 55ms/step - loss: 0.6926 - acc: 0.5095 - val_loss: 0.6925 - val_acc: 0.5306
Epoch 3/10
23/23 [==============================] - 1s 55ms/step - loss: 0.6902 - acc: 0.5518 - val_loss: 0.6918 - val_acc: 0.5265
Epoch 4/10
23/23 [==============================] - 1s 64ms/step - loss: 0.6749 - acc: 0.6144 - val_loss: 0.6867 - val_acc: 0.5306
Epoch 5/10
23/23 [==============================] - 1s 61ms/step - loss: 0.6049 - acc: 0.7520 - val_loss: 0.6477 - val_acc: 0.6041
Epoch 6/10
23/23 [==============================] - 1s 61ms/step - loss: 0.4025 - acc: 0.8910 - val_loss: 0.6414 - val_acc: 0.6327
Epoch 7/10
23/23 [==============================] - 1s 55ms/step - loss: 0.2533 - acc: 0.9278 - val_loss: 0.6653 - val_acc: 0.6082
Epoch 8/10
23/23 [==============================] - 1s 61ms/step - loss: 0.1653 - 

# Word2Vec LSTM

In [108]:
from gensim.models import Word2Vec
import numpy as np

X_sentences = [sentence.split() for sentence in X]

word2vec_model = Word2Vec(X_sentences, vector_size=32, min_count=1)

word2vec_dict = {word: word2vec_model.wv[word] for word in word2vec_model.wv.index_to_key}

embedding_matrix = np.zeros((5000, 32))

for word, i in tokenizer.word_index.items():
    embedding_vector = word2vec_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(5000, 32, weights=[embedding_matrix]))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 7s 104ms/step - loss: 0.6936 - acc: 0.4918 - val_loss: 0.6926 - val_acc: 0.5306
Epoch 2/10
23/23 [==============================] - 1s 44ms/step - loss: 0.6924 - acc: 0.5082 - val_loss: 0.6928 - val_acc: 0.5102
Epoch 3/10
23/23 [==============================] - 1s 46ms/step - loss: 0.6899 - acc: 0.5858 - val_loss: 0.6911 - val_acc: 0.5429
Epoch 4/10
23/23 [==============================] - 1s 45ms/step - loss: 0.6711 - acc: 0.6717 - val_loss: 0.6769 - val_acc: 0.5837
Epoch 5/10
23/23 [==============================] - 1s 45ms/step - loss: 0.5608 - acc: 0.8106 - val_loss: 0.6398 - val_acc: 0.6367
Epoch 6/10
23/23 [==============================] - 1s 54ms/step - loss: 0.3342 - acc: 0.9169 - val_loss: 0.7003 - val_acc: 0.5796
Epoch 7/10
23/23 [==============================] - 1s 45ms/step - loss: 0.2062 - acc: 0.9469 - val_loss: 0.6866 - val_acc: 0.6122
Epoch 8/10
23/23 [==============================] - 1s 46ms/step - loss: 0.1284 - 

# Glove

In [114]:
import numpy as np

embeddings_index = {}
with open('C:/Users/Dell/Desktop/glove.6B/glove.6B.100d.txt') as G:
    for line in G:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((5000, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(5000, 100, weights=[embedding_matrix], input_length=50, trainable=False))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_seq_padded, y_train, validation_data=(X_test_seq_padded, y_test), epochs=5)

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)


Epoch 1/5
23/23 [==============================] - 6s 115ms/step - loss: 0.6951 - accuracy: 0.4823 - val_loss: 0.6901 - val_accuracy: 0.5306
Epoch 2/5
23/23 [==============================] - 2s 79ms/step - loss: 0.6918 - accuracy: 0.5068 - val_loss: 0.6905 - val_accuracy: 0.5265
Epoch 3/5
23/23 [==============================] - 2s 76ms/step - loss: 0.6904 - accuracy: 0.4714 - val_loss: 0.6901 - val_accuracy: 0.4857
Epoch 4/5
23/23 [==============================] - 2s 72ms/step - loss: 0.6894 - accuracy: 0.5245 - val_loss: 0.6906 - val_accuracy: 0.4898
Epoch 5/5
8/8 [==============================] - 1s 21ms/step
Accuracy: 0.4857142857142857
Precision: 0.47577092511013214
F1-score: 0.631578947368421
Recall: 0.9391304347826087


# FastText

In [115]:
from gensim.models import FastText
X_sentences = [sentence.split() for sentence in X]

fasttext_model = FastText(X_sentences, vector_size=32, min_count=1)

fasttext_dict = {word: fasttext_model.wv[word] for word in fasttext_model.wv.index_to_key}

embedding_matrix = np.zeros((5000, 32))

for word, i in tokenizer.word_index.items():
    embedding_vector = fasttext_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(5000, 32, weights=[embedding_matrix]))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_seq_padded, y_train, epochs=10, validation_data=(X_test_seq_padded, y_test))

y_pred = model.predict(X_test_seq_padded)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

Epoch 1/10
23/23 [==============================] - 10s 107ms/step - loss: 0.6936 - acc: 0.4877 - val_loss: 0.6933 - val_acc: 0.4694
Epoch 2/10
23/23 [==============================] - 1s 56ms/step - loss: 0.6931 - acc: 0.5054 - val_loss: 0.6929 - val_acc: 0.5592
Epoch 3/10
23/23 [==============================] - 1s 48ms/step - loss: 0.6928 - acc: 0.5000 - val_loss: 0.6926 - val_acc: 0.5633
Epoch 4/10
23/23 [==============================] - 1s 49ms/step - loss: 0.6911 - acc: 0.5545 - val_loss: 0.6926 - val_acc: 0.5061
Epoch 5/10
23/23 [==============================] - 1s 54ms/step - loss: 0.6852 - acc: 0.6335 - val_loss: 0.6890 - val_acc: 0.5347
Epoch 6/10
23/23 [==============================] - 1s 60ms/step - loss: 0.6437 - acc: 0.7275 - val_loss: 0.6855 - val_acc: 0.5306
Epoch 7/10
23/23 [==============================] - 1s 46ms/step - loss: 0.5049 - acc: 0.8311 - val_loss: 0.6431 - val_acc: 0.6204
Epoch 8/10
23/23 [==============================] - 1s 46ms/step - loss: 0.3183 -

# ELMO

In [119]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.load('https://tfhub.dev/google/elmo/3')

def elmo_vectors(text):
    embeddings = elmo(signature='default')['elmo']
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        return sess.run(embeddings)

X_train_vec = elmo_vectors(X_train)
X_test_vec = elmo_vectors(X_test)

model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(None, 1024)))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train_vec, y_train, epochs=10, validation_data=(X_test_vec, y_test))

y_pred = model.predict(X_test_vec)

threshold = 0.5
y_pred = [1 if p > threshold else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("Recall:", recall)

ModuleNotFoundError: No module named 'tensorflow_hub'